# Answering Business Questions Using SQL

This is a guided project from dataquest.io  
The project will use data from a fictional music sales company.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


[('album', 'table'),
 ('artist', 'table'),
 ('customer', 'table'),
 ('employee', 'table'),
 ('genre', 'table'),
 ('invoice', 'table'),
 ('invoice_line', 'table'),
 ('media_type', 'table'),
 ('playlist', 'table'),
 ('playlist_track', 'table'),
 ('track', 'table')]

### Best genre for the USA
I am asked to write a query to decide which three new artists to include based on sales in the USA.  

Regal - Hip-Hop  
Red Tone - Punk  
Meteor and the Girls - Pop  
Slim Jim Bites - Blues  

In [3]:
%%sql
WITH usa_sales AS
( SELECT 
        il.* FROM invoice_line il
        INNER JOIN invoice i ON i.invoice_id = il.invoice_id
        INNER JOIN customer c ON c.customer_id = i.customer_id
        WHERE c.country = "USA")
SELECT g.name genre, 
    COUNT(us.invoice_line_id) total_sales, 
    (CAST(COUNT(us.invoice_line_id) AS FLOAT) / (SELECT COUNT(*) FROM usa_sales)) * 100 percent_sales
FROM usa_sales us
INNER JOIN track t ON t.track_id = us.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY g.name;

Done.


[('Alternative', 35, 3.3301617507136063),
 ('Alternative & Punk', 130, 12.369172216936251),
 ('Blues', 36, 3.425309229305423),
 ('Classical', 4, 0.3805899143672693),
 ('Easy Listening', 13, 1.236917221693625),
 ('Electronica/Dance', 5, 0.47573739295908657),
 ('Heavy Metal', 3, 0.285442435775452),
 ('Hip Hop/Rap', 20, 1.9029495718363463),
 ('Jazz', 14, 1.3320647002854424),
 ('Latin', 22, 2.093244529019981),
 ('Metal', 124, 11.798287345385347),
 ('Pop', 22, 2.093244529019981),
 ('R&B/Soul', 53, 5.042816365366318),
 ('Reggae', 6, 0.570884871550904),
 ('Rock', 561, 53.37773549000951),
 ('Soundtrack', 2, 0.19029495718363465),
 ('TV Shows', 1, 0.09514747859181732)]

Rock is the mot popular genre sold in the US. However the four genres to choose from are: Hip-Hop, Punk, Pop and Blues.  
12% sales for alternative & punk makes punk a clear choice. Hip-hop has the least sales with 1.9%, but only marginally higher than pop at 2.0%.  
Based on the sales alone the three genres to pick are: Punk, Blues and Pop.

### Employee with the most sales
Now I am asked to find out which employee had the most sales and any other characrteristics which may help business decisions

In [8]:
%%sql
SELECT e.employee_id,
        e.first_name || " " || e.last_name employee_name,
        e.country employee_country,
        e.hire_date,
        COUNT(c.customer_id) num_customers,
        SUM(i.total) total_sales
FROM employee e
INNER JOIN customer c ON c.support_rep_id = e.employee_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY employee_name;

Done.


[(3, 'Jane Peacock', 'Canada', '2017-04-01 00:00:00', 212, 1731.5100000000039),
 (4, 'Margaret Park', 'Canada', '2017-05-03 00:00:00', 214, 1584.0000000000034),
 (5, 'Steve Johnson', 'Canada', '2017-10-17 00:00:00', 188, 1393.920000000002)]

Steve Johnson has the least sales and the least number of customers. However Steve started 5 months after Margaret and bearly 6 months after Jane, which explains the lack of sales.  
Jane, who has the most sales was the first sales person. Interestingly Jane has 2 less customers than Margaret, but around 150 more in sales. Which means Jane has more repeat custom.

### Sales by country
I have now been asked to find out the sales by country

In [21]:
%%sql
WITH country_other AS
    (
    SELECT
            CASE
                WHEN(
                    SELECT COUNT(*)
                    FROM customer
                    WHERE country = c.country
                ) = 1 THEN "Other"
                    ELSE c.country
            END AS country,
            c.customer_id
            FROM customer c
    )

SELECT country_other.country,
                COUNT(DISTINCT country_other.customer_id) num_customers,
        SUM(i.total) total_sales,
        CAST(SUM(i.total) AS FLOAT)/COUNT(DISTINCT country_other.customer_id) sales_per_customer,
        CAST(SUM(i.total) AS FLOAT)/COUNT(DISTINCT i.invoice_id) sales_per_order,
        CASE WHEN country_other.country = "Other" THEN 1 ELSE 0 END AS sort
FROM country_other
INNER JOIN invoice i ON i.customer_id = country_other.customer_id
GROUP BY country_other.country
ORDER BY sort asc, total_sales desc;

Done.


[('USA', 13, 1040.4899999999998, 80.0376923076923, 7.942671755725189, 0),
 ('Canada', 8, 535.5900000000001, 66.94875000000002, 7.047236842105265, 0),
 ('Brazil', 5, 427.68000000000006, 85.53600000000002, 7.0111475409836075, 0),
 ('France', 5, 389.0699999999999, 77.81399999999998, 7.781399999999998, 0),
 ('Germany', 4, 334.62, 83.655, 8.161463414634147, 0),
 ('Czech Republic', 2, 273.24000000000007, 136.62000000000003, 9.108000000000002, 0),
 ('United Kingdom', 3, 245.52, 81.84, 8.768571428571429, 0),
 ('Portugal', 2, 185.13000000000002, 92.56500000000001, 6.383793103448276, 0),
 ('India', 2, 183.14999999999998, 91.57499999999999, 8.72142857142857, 0),
 ('Other', 15, 1094.9400000000003, 72.99600000000002, 7.44857142857143, 1)]

### Purchases of tracks v albums